In [3]:
import numpy as np
import pandas as pd
loan=pd.read_csv('loan_result.csv')
loan_group=loan.dropna().groupby('application_id').sum()
loan_group['num']=1
loan_group.groupby('is_applied').sum()[['num']]

,num
is_applied,
0.0,351356
1.0,267622
2.0,71144
3.0,23310
4.0,8721
5.0,3593
6.0,1590
7.0,717
8.0,387


In [1]:
import pandas as pd
data = pd.read_csv("finaljoined.csv")

C:\Users\dofod\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
del data["Unnamed: 0"]

In [3]:
data.loanapply_insert_time = pd.to_datetime(data.loanapply_insert_time)
x = data.birth_year.astype(str)
x = x.str[:4]
data.birth_year = pd.to_datetime(x, format = "%Y")
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)

In [4]:
import numpy as np
from catboost import CatBoost, Pool

In [5]:
income_dummy=pd.get_dummies(data['income_type'])

In [6]:
data=pd.concat([data,income_dummy],axis=1)

In [7]:
del data['income_type']

In [8]:
employment_dummy=pd.get_dummies(data['employment_type'])

data=pd.concat([data,employment_dummy],axis=1)

del data['employment_type']

In [9]:
house_dummy=pd.get_dummies(data['houseown_type'])

data=pd.concat([data,house_dummy],axis=1)

del data['houseown_type']

In [10]:
purpose_dummy=pd.get_dummies(data['purpose'])
purpose_dummy.rename(columns={"기타":"기타목적"},inplace=True)

data=pd.concat([data,purpose_dummy],axis=1)

del data['purpose']

In [25]:
train_data=data[data['insert_time']<'2022-05-16']

In [26]:
train_data2=train_data.copy()

In [27]:
del train_data['is_applied']

In [28]:
test_data=data[data['insert_time']>='2022-05-16']
test_data=test_data[test_data['insert_time']<'2022-06-01']

In [29]:
test_data2=test_data.copy()

In [30]:
del test_data['is_applied']

In [31]:
train_label=train_data2['is_applied']

In [32]:
test_label=test_data2['is_applied']

In [33]:
from catboost import CatBoostClassifier

In [34]:
model = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [35]:
model.fit(train_data,train_label)

0:	learn: 0.6653837	total: 298ms	remaining: 298ms
1:	learn: 0.6608896	total: 437ms	remaining: 0us


In [36]:
preds_proba=model.predict_proba(test_data)

In [40]:
preds_proba

array([[0.55797681, 0.44202319],
       [0.55797681, 0.44202319],
       [0.39899535, 0.60100465],
       ...,
       [0.80182478, 0.19817522],
       [0.35458262, 0.64541738],
       [0.35458262, 0.64541738]])

In [41]:
preds_proba=preds_proba[:,1]

In [42]:
preds_proba

array([0.44202319, 0.44202319, 0.60100465, ..., 0.19817522, 0.64541738,
       0.64541738])

In [37]:
from sklearn.metrics import f1_score

In [45]:
f1_score(test_label,preds_proba>0.5179389239664024)

0.6468902814263878

In [46]:
preds_class=model.predict(test_data)

In [47]:
preds_class

array([0., 0., 1., ..., 0., 1., 1.])

In [48]:
f1_score(test_label,preds_class)

0.6530162579366632

In [15]:
data=data[data['insert_time']<'2022-06-01']

In [18]:
data=data.sort_values(by='is_applied')

In [22]:
data.drop_duplicates(['application_id'],keep='last')

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,LIVING,SWITCHLOAN,기타목적,대환대출,사업자금,생활비,자동차구입,전월세보증금,주택구입,투자
8875937,575158,2022-03-23 11:32:59,32,56,5000000.0,17.1,0.0,269993,1977-01-01,0.0,...,0,0,0,0,0,1,0,0,0,0
8875827,232320,2022-03-23 19:11:35,59,150,10000000.0,16.3,0.0,621825,1975-01-01,0.0,...,0,0,0,0,0,0,1,0,0,0
8875897,702633,2022-03-23 21:00:49,38,223,9000000.0,18.4,0.0,118733,1983-01-01,1.0,...,0,0,0,0,0,1,0,0,0,0
8875898,856359,2022-03-23 16:10:43,38,223,9000000.0,18.4,0.0,118733,1983-01-01,1.0,...,0,0,0,0,0,1,0,0,0,0
8875899,238027,2022-03-23 21:03:11,17,19,12000000.0,18.0,0.0,118733,1983-01-01,1.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7546752,1705221,2022-03-02 17:16:45,15,141,35000000.0,15.4,1.0,687493,1967-01-01,1.0,...,0,0,0,0,0,1,0,0,0,0
8081932,641168,2022-03-15 10:43:34,11,170,29000000.0,14.2,1.0,290981,1962-01-01,1.0,...,0,0,0,0,0,1,0,0,0,0
7546754,1440638,2022-03-02 13:26:16,52,187,3000000.0,6.0,1.0,244517,1962-01-01,0.0,...,0,0,0,0,0,1,0,0,0,0
1457755,1486409,2022-04-28 14:24:10,4,268,45000000.0,13.1,1.0,473029,1997-01-01,1.0,...,0,0,0,0,0,1,0,0,0,0


In [23]:
del data['loanapply_insert_time']
del data['bank_id']
del data['product_id']
del data['loan_limit']
del data['loan_rate']

In [24]:
data

,application_id,is_applied,user_id,birth_year,gender,insert_time,credit_score,yearly_income,company_enter_month,desired_amount,...,LIVING,SWITCHLOAN,기타목적,대환대출,사업자금,생활비,자동차구입,전월세보증금,주택구입,투자
8875937,575158,0.0,269993,1977-01-01,0.0,2022-03-23 11:32:57,580.0,1000000.0,2022-06-01,5000000.0,...,0,0,0,0,0,1,0,0,0,0
8875827,232320,0.0,621825,1975-01-01,0.0,2022-03-23 19:11:35,540.0,1000000.0,2022-02-01,20000000.0,...,0,0,0,0,0,0,1,0,0,0
8875897,702633,0.0,118733,1983-01-01,1.0,2022-03-23 21:00:48,630.0,36000000.0,2015-11-01,50000000.0,...,0,0,0,0,0,1,0,0,0,0
8875898,856359,0.0,118733,1983-01-01,1.0,2022-03-23 16:10:43,630.0,36000000.0,2015-11-01,10000000.0,...,0,0,0,0,0,1,0,0,0,0
8875899,238027,0.0,118733,1983-01-01,1.0,2022-03-23 21:03:10,630.0,36000000.0,2015-11-01,10000000.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7546752,1705221,1.0,687493,1967-01-01,1.0,2022-03-02 17:16:43,640.0,25000000.0,2022-06-01,50000000.0,...,0,0,0,0,0,1,0,0,0,0
8081932,641168,1.0,290981,1962-01-01,1.0,2022-03-15 10:43:18,570.0,31000000.0,2021-11-01,10000000.0,...,0,0,0,0,0,1,0,0,0,0
7546754,1440638,1.0,244517,1962-01-01,0.0,2022-03-02 13:26:01,890.0,28000000.0,2021-01-01,3000000.0,...,0,0,0,0,0,1,0,0,0,0
1457755,1486409,1.0,473029,1997-01-01,1.0,2022-04-28 14:24:08,611.0,30.0,2021-12-01,5.0,...,0,0,0,0,0,1,0,0,0,0


In [44]:
data['is_applied'].mean()

0.5179389239664024